# FITC

In [1]:
import sys
# Add the symgp folder path to the sys.path list
module_path = r'/Users/jaduol/Documents/Uni (original)/Part II/IIB/MEng Project/'
if module_path not in sys.path:
    sys.path.append(module_path)

from symgp import SuperMatSymbol, utils, MVG, Variable, SuperDiagMat, Kernel
from sympy import symbols, ZeroMatrix, Identity
from IPython.display import display, Math, Latex

In [2]:
import imp
imp.reload(utils)
imp.reload(kernel)

NameError: name 'kernel' is not defined

In [2]:
m, n, l = symbols('m n l')
s_y = symbols('\u03c3_y')
K = Kernel()

## 1. Induding prior

In [3]:
u = Variable('u',m,1)
p_u = MVG([u],mean=ZeroMatrix(m,1),cov=K(u,u))

print("p_u:")
display(Latex(utils.matLatex(p_u)))

p_u:


<IPython.core.display.Latex object>

## 2. Training and Test Conditionals

In [4]:
f, fs, y = utils.variables("f f_{*} y",[n, l, n])

q_fgu = MVG([f], mean=K(f,u)*K(u,u).I*u,
                 cov=SuperDiagMat(K(f,f)-K(f,u)*K(u,u).I*K(u,f)),
                 cond_vars=[u],
                 prefix='q_{FITC}')

print("q_fgu:")
display(Latex(utils.matLatex(q_fgu)))

q_fgu:


<IPython.core.display.Latex object>

In [5]:
q_fsgu = MVG([fs], mean=K(fs,u)*K(u,u).I*u,
                   cov=K(fs,fs)-K(fs,u)*K(u,u).I*K(u,fs),
                   cond_vars=[u],
                   prefix='q_{FITC}')

print("q_fgu:")
display(Latex(utils.matLatex(q_fsgu)))

q_fgu:


<IPython.core.display.Latex object>

In [6]:
# q(f,fs|u)
q_f_fs_g_u = q_fgu*q_fsgu

print("q_f_fs_g_u:")
display(Latex(utils.matLatex(q_f_fs_g_u)))

q_f_fs_g_u:


<IPython.core.display.Latex object>

In [7]:
# q(f,fs,u)
q_f_fs_u = q_f_fs_g_u*p_u

print("q_f_fs_u:")
display(Latex(utils.matLatex(q_f_fs_u)))

q_f_fs_u:


<IPython.core.display.Latex object>

In [8]:
# Effective prior: q(f,fs)
q_f_fs = q_f_fs_u.marginalise([u])

print("q_f_fs:")
display(Latex(utils.matLatex(q_f_fs)))

q_f_fs:


<IPython.core.display.Latex object>

## 3. Likelihood

In [9]:
p_ygf = MVG([y],mean=f,cov=s_y**2*Identity(n),cond_vars=[f])

print("p_ygf:")
display(Latex(utils.matLatex(p_ygf)))

p_ygf:


<IPython.core.display.Latex object>

## 4. Inference 

In [10]:
# q(f,fs,y)
q_f_fs_y = p_ygf*q_f_fs

print("q_f_fs_y:")
display(Latex(utils.matLatex(q_f_fs_y)))

q_f_fs_y:


<IPython.core.display.Latex object>

In [11]:
# q(f,fs|y)
q_f_fs_g_y = q_f_fs_y.condition([y])

print("q_f_fs_g_y:")
display(Latex(utils.matLatex(q_f_fs_g_y)))

q_f_fs_g_y:


<IPython.core.display.Latex object>

In [12]:
# q(fs|y)
q_fs_g_y = q_f_fs_g_y.marginalise([f])

print("q_fs_g_y:")
display(Latex(utils.matLatex(q_fs_g_y)))

q_fs_g_y:


<IPython.core.display.Latex object>

## Example of Kernel substitution

In [13]:
import imp
imp.reload(utils)
from symgp import Covariance
M = Covariance(u, name='M_{u,u}', full_expr=K(u,u).I)
Q = Kernel(name='Q', sub_kernels=[K,K], kernel_type='mul', mat=M)
#repl_dict = {K(fs,u)*K(u,u).I*K(u,f): Q(fs,f),
#             K(f,u)*K(u,u).I*K(u,f): Q(f,f),
#             K(f,u)*K(u,u).I*K(u,fs): Q(f,fs)}
repl_dict = [Q]

q_fs_g_y.mean.expanded = utils.replace(q_fs_g_y.mean.to_full_expr(), repl_dict)
q_fs_g_y.covar.expanded = utils.replace(q_fs_g_y.covar.to_full_expr(), repl_dict)


subbed_expr:  Q_{f_{*},f}*(σ_y**2*I + diag[K_{f,f} + (-1)*Q_{f,f}] + Q_{f,f})^-1*y
srepr(subbed_expr):  SuperMatMul(KernelMatrix('Q_{f_{*},f}', Symbol('l'), Symbol('n')), Inverse(SuperMatAdd(MatMul(Pow(Symbol('σ_y'), Integer(2)), Identity(Symbol('n'))), SuperDiagMat(SuperMatAdd(KernelMatrix('K_{f,f}', Symbol('n'), Symbol('n')), SuperMatMul(Integer(-1), KernelMatrix('Q_{f,f}', Symbol('n'), Symbol('n'))))), SuperMatMul(KernelMatrix('Q_{f,f}', Symbol('n'), Symbol('n'))))), Variable('y', Symbol('n'), Integer(1)))
subbed_expr:  K_{f_{*},f_{*}} + (-1)*Q_{f_{*},f}*(σ_y**2*I + diag[K_{f,f} + (-1)*Q_{f,f}] + Q_{f,f})^-1*Q_{f,f_{*}}
srepr(subbed_expr):  SuperMatAdd(KernelMatrix('K_{f_{*},f_{*}}', Symbol('l'), Symbol('l')), SuperMatMul(Integer(-1), KernelMatrix('Q_{f_{*},f}', Symbol('l'), Symbol('n')), Inverse(SuperMatAdd(MatMul(Pow(Symbol('σ_y'), Integer(2)), Identity(Symbol('n'))), SuperDiagMat(SuperMatAdd(KernelMatrix('K_{f,f}', Symbol('n'), Symbol('n')), SuperMatMul(Integer(-1), KernelMatrix(

In [14]:
imp.reload(utils)

print("q_fs_g_y (replaced):")
display(Latex(utils.matLatex(q_fs_g_y)))

q_fs_g_y (replaced):


<IPython.core.display.Latex object>